In [1]:
import json
import glob
import os
import datetime
import re

In [2]:
from slack_export import SlackExport, normalize_links

In [10]:
# data_folder = '/Users/alex/Documents/ODS/oct_4_2016_dump'
data_folder = './ODS_dump_Mar_10_2017'
ods = SlackExport(data_folder)

## Message types

In [14]:
import pandas as pd

In [15]:
df_msg = pd.DataFrame.from_records(ods.messages)

In [16]:
df_msg.fillna(0).groupby(['type', 'subtype'])['dt'].count()

type     subtype          
message  0                    110721
         bot_add                  10
         bot_message             745
         bot_remove                6
         channel_archive          19
         channel_join          24227
         channel_leave           761
         channel_name             28
         channel_purpose          61
         channel_topic            56
         channel_unarchive         1
         file_comment             24
         file_mention             11
         file_share             1202
         me_message                2
         pinned_item              88
Name: dt, dtype: int64

## Tf-idf

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
all_texts = [
    normalize_links(msg['text'])
    for msg in ods.messages
    if msg['type'] == 'message' and msg.get('subtype') is None
]

In [28]:
from random import sample

In [84]:
anybody = '?'.decode('utf-8')

In [85]:
filtered = filter(lambda x: anybody in x.lower() ,all_texts)

len(filtered)

22396

In [87]:
for _msg in sample(filtered, 10):
    print(_msg)
    print('-'*30)

почему? Другими словами вы разрешаете делать разные монотонные преобразования над фичами и утверждаете, что порядок сплитов и по каким переменным каждый сплит сохранится?   Хотелось бы почитать доказательство. Если на шаге i выбрана таже фича для сплита, что в исходном дереве, то вопросов нет. Но почему будет выбрана имена она?
------------------------------
@U041SH27M   ну так не понял, где искать вас. в субботу тогда? ;)
------------------------------
текст слева, текст справа -- это текст одной категории и текст другой категории?
------------------------------
Как так?
------------------------------
@U041SH27M: а чем не аналитика? Вполне себе. Deep learning в том же маркетинге никому не упал.
------------------------------
Можете проверить звук сейчас норм?
------------------------------
А что такое "колметоды"?
------------------------------
тут да, отличие всего в 2 символа. А если так:
"СуШ1(а БХК ЧЕЛНОЧОК 400(. ,, ПЛ/ПАК" и "сушка бхк челночок 400г пл/пак"?
---------------------

In [53]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import EnglishStemmer, RussianStemmer

ru_stemmer = RussianStemmer()
en_stemmer = EnglishStemmer()
analyzer = CountVectorizer().build_analyzer()

def is_russian_word(w):
    if not isinstance(w, unicode):
        w = w.decode('utf8')
    for c in w:
        if ord(c) >= ord(u'а') and ord(c) <= ord(u'я'):
            return True
    return False

def stemmed_words(doc):
    for w in analyzer(doc):
        yield ru_stemmer.stem(w) if is_russian_word(w) else en_stemmer.stem(w)

In [54]:
from tqdm import tqdm

In [55]:
%%time
vectorizer = TfidfVectorizer(min_df=5, analyzer=stemmed_words).fit(all_texts)

CPU times: user 2min 9s, sys: 1.79 s, total: 2min 11s
Wall time: 2min 11s


In [67]:
from __future__ import print_function

In [70]:
from operator import itemgetter

In [74]:
vectorizer?

In [79]:
for _item in sample(vectorizer.stop_words_,100):
    print(_item,end='\t')

north	брр	разработок	прерыван	100014	малахов	0e90e3f79329	отправлен	pedobear	c1h4gglmr	research_discussion_stacked_approxim	badass	blas_mkl_info	richrelev	заболеют	востряк	малоизвестн	колокейт	13085	недосуг	elections2011	izbirkom2	0o8	присоветова	спим	ytn	колдовств	216	jlevi	метеоризм	flavor	вентилирован	гадан	вяж	bmstu	dietrich	main_en	rtree	burger	597s110111n	вашингтонск	codalabtemp	стекловат	навчан	сенсац	малац	накрыт	kuroyanagi	нестра	математичненьк	функшенс	подбега	нажра	угорет	жиробас	зашит	гитлаб	сосдеств	x_max_depth	ltr	p1469619524000109	саутсорс	pllb7e2g7aspssa_plfewnd6	ff77cc0a8d29	орегон	fandom	аптек	подсос	produkte_rootserv	лицензион	понын	tjurjuwbb	отягча	друид	третьекурсниц	скопипастнул	cjlin	бейт	complete_separation_logit_model	перелива	j0zpiiso	231699	beefsteak	забив	numa	нетехническ	200gib	хахах	dailydot	35181	трэч	1842	телескопическ	_________________________________	канистр	emeraldgrouppublish	ветхост	permut	настольник	tg1mumpwvrml6qd0	

In [72]:
for _word, _count in sorted(vectorizer.vocabulary_.items(),key=itemgetter(1),reverse=True)[:20]:
    print(_word,end=' ')
    print(_count)

ящик 14900
ячейк 14899
ячеек 14898
яхт 14897
ясност 14896
ясн 14895
ярмарк 14894
ярлык 14893
яркост 14892
ярк 14891
ярд 14890
японск 14889
япон 14888
яп 14887
яндексоид 14886
яндексовск 14885
яндексов 14884
яндекс 14883
январ 14882
ян 14881


In [57]:
msg

{'channel': u'C1CFSK1NW',
 'dt': datetime.datetime(2016, 6, 6, 18, 15, 37, 3),
 u'members': [u'U04CH4QBD', u'U09BY2N3X', u'U0JNAUT99'],
 u'subtype': u'channel_archive',
 u'text': u'<@U04CH4QBD|ermakovpetr> archived the channel (w/ 3 members)',
 u'ts': 1465226137.000003,
 u'type': u'message',
 u'user': u'U04CH4QBD'}

In [58]:
all_texts = [
    normalize_links(msg['text'])
    for msg in ods.messages
    if msg['type'] == 'message' and msg.get('subtype') is None
]

## Reaction classifier

In [163]:
react_texts = []
react_counts = []

for msg in ods.messages:
    if msg['type'] == 'message' and msg.get('subtype') is None:
        react_texts.append(normalize_links(msg['text']))
        
        msg_reacts = {}
        for record in msg.get('reactions', []):
            msg_reacts[record['name']] = record['count']
        react_counts.append(msg_reacts)

In [192]:
import collections
unique_reacts = collections.Counter()
for c in react_counts:
    for r in c:
        unique_reacts[r] += 1

In [199]:
selected_reacts = [r for r, c in unique_reacts.most_common(20)]

In [164]:
X = vectorizer.transform(react_texts)

In [186]:
import numpy as np
from sklearn.linear_model import LogisticRegression

In [223]:
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

In [229]:
y.max()

86

In [231]:
model_react = {}
for react_target in selected_reacts:
    y = np.array([int(c.get(react_target, 0) > 0) for c in react_counts])
    model_react[react_target] = LogisticRegression().fit(X, y)

    cv = StratifiedShuffleSplit(y, random_state=123, test_size=.15, train_size=.85, n_iter=3)
    print '%s, positives %.2f%%, cv auc %.5f' % (
        react_target,
        y.mean()*100,
        cross_val_score(LogisticRegression(), X, y, cv=cv, scoring='roc_auc').mean(),
    )

+1, positives 3.15%, cv auc 0.74158
heavy_plus_sign, positives 1.57%, cv auc 0.66345
joy, positives 0.50%, cv auc 0.70246
trollface, positives 0.34%, cv auc 0.65444
thinking_face, positives 0.34%, cv auc 0.66133
metal, positives 0.28%, cv auc 0.77424
100, positives 0.24%, cv auc 0.63487
parrot, positives 0.21%, cv auc 0.73555
muscle, positives 0.21%, cv auc 0.75717
but_why, positives 0.20%, cv auc 0.67768
eyes, positives 0.20%, cv auc 0.55336
aussieparrot, positives 0.20%, cv auc 0.71491
putin, positives 0.19%, cv auc 0.69807
white_check_mark, positives 0.17%, cv auc 0.59538
slava, positives 0.16%, cv auc 0.74864
facepalm, positives 0.13%, cv auc 0.62229
feelsgood, positives 0.13%, cv auc 0.56028
levenchuk, positives 0.13%, cv auc 0.71326
ban, positives 0.11%, cv auc 0.70853
youknow, positives 0.11%, cv auc 0.57877


In [232]:
def top_reactions(text):
    x = vectorizer.transform([text])
    preds = [
        (r, model_react[r].predict_proba(x)[0, 1])
        for r in selected_reacts
    ]
    preds.sort(key=lambda (r, p): p, reverse=True)
    return preds[:5]

In [233]:
top_reactions(u'Можно. Тут пара человек угорала с истории про козла')

[(u'+1', 0.031918339825498104),
 (u'heavy_plus_sign', 0.014286038840257299),
 (u'thinking_face', 0.0046909572491761276),
 (u'trollface', 0.0041309662885016096),
 (u'slava', 0.0039838725771864291)]

In [234]:
top_reactions(u'И мемесы у нас русские')

[(u'+1', 0.019815695123274447),
 (u'heavy_plus_sign', 0.011856093208197686),
 (u'putin', 0.010068268864851336),
 (u'trollface', 0.0093261077915930933),
 (u'joy', 0.0065046704319646226)]

In [235]:
top_reactions(u'русских у нас и так хватает')

[(u'heavy_plus_sign', 0.018157298143154872),
 (u'+1', 0.013539816190672476),
 (u'trollface', 0.0082548602457012139),
 (u'joy', 0.0065746048791814034),
 (u'putin', 0.0051477862065029394)]

In [236]:
top_reactions(u'Есть ли здесь люди, которые работают DS в аптечных сетях? Можете ли вы привести примеры конкретных задач, решенных вами и которые имели конкретный результат, выразившийся в увеличении продаж или снижении затрат?')

[(u'+1', 0.038385176875187284),
 (u'heavy_plus_sign', 0.01267402335827319),
 (u'thinking_face', 0.0039334486914132088),
 (u'metal', 0.0035588699999281213),
 (u'100', 0.0030812867555780484)]

In [237]:
top_reactions(u'Т.е. на самом деле это не они ему придумали, а он такой "ребят своим говором вы называете меня зефиркой, давайте-ка лучше придумаем немецкое имя"')

[(u'heavy_plus_sign', 0.05352472707854166),
 (u'+1', 0.029715544162347621),
 (u'thinking_face', 0.0073329487286867365),
 (u'joy', 0.0049451150425887276),
 (u'trollface', 0.0031770037231480318)]

## Channel classifier

In [59]:
channel_messages_cnt = collections.Counter(channel_labels)
selected_channels = [r for r, c in channel_messages_cnt.most_common(50)]
channel_messages_cnt.most_common(50)

NameError: name 'collections' is not defined

In [251]:
channel_texts = []
channel_labels = []

for msg in ods.messages:
    if msg['type'] == 'message' and msg.get('subtype') is None and ods.channels[msg['channel']]['name'] in selected_channels:
        channel_texts.append(normalize_links(msg['text']))
        channel_labels.append(ods.channels[msg['channel']]['name'])
        
X_channel = vectorizer.transform(channel_texts)

In [ ]:
from sklearn.metrics import average_precision_score

In [269]:
for train_indices, test_indices in cv:
    model = LogisticRegression().fit(X_channel[train_indices, :], np.array(channel_labels)[train_indices])
    y_true = np.vstack([
        [int(ch == l) for ch in model.classes_]
        for l in channel_labels
    ])
    sc = average_precision_score(y_true[test_indices, :], model.predict_proba(X_channel[test_indices, :]))
    print sc

0.181920245541
0.187385121118
0.189705231823


In [271]:
model_channel = LogisticRegression().fit(X_channel, np.array(channel_labels))

In [272]:
def predict_channel(text):
    X = vectorizer.transform([text])
    y_pred = model_channel.predict_proba(X)
    preds = zip(model_channel.classes_, y_pred[0, :])
    preds.sort(key=lambda (c, p): p, reverse=True)
    return preds[:5]

In [273]:
predict_channel(u'подскажите пожалуйста как запустить факторизационные машины')

[(u'theory_and_practice', 0.16790929041550848),
 (u'python', 0.15014067919781149),
 (u'_random_flood', 0.087561429305589536),
 (u'_general', 0.071414662006578311),
 (u'big_data', 0.071126243710510059)]

In [274]:
predict_channel(u'Слайды с семинара в ИППИ на прошлой неделе на тему "Математические модели временных рядов с трендом в задачах обнаружения разладки"')

[(u'_meetings', 0.44959758801100247),
 (u'theory_and_practice', 0.19180315910086793),
 (u'_random_flood', 0.046230132693773567),
 (u'conference', 0.026185563394516838),
 (u'_general', 0.024911607092178777)]

In [275]:
predict_channel(u'интересно, почему возраст - обязательное поле при регистрации')

[(u'_random_flood', 0.20119753387775735),
 (u'_meetings', 0.084453090868886332),
 (u'_general', 0.083119532724364092),
 (u'python', 0.067297056103339165),
 (u'mltrainings_beginners', 0.064526763286998134)]

In [277]:
predict_channel(u'''
Всем привет

В компании Criteo есть интересные позиции Data Scientist и Machine Learning Engineer разных уровней. В основном в Европе (Париж, Барселона, Амстердам, Лондон и др.), но есть также в США и Азии.

Например, есть позиция Sr. Data Analyst, где надо анализировать покупательную заинтересованность людей в оффлайн магазинах и матчить ее с данными в онлайн. Оффлайн данные поступают с маячков (iBeacons).

Есть несколько других позиций связанные с улучшением алгоритмов движка Criteo, рекомендательной системой и обработки больших данных.

Позиции full-time, on-site. Компания помогает с переездом (visa + relocation bonus).

Вот видео как круто в Criteo: https://youtu.be/i1zC3H3_gik

Если кого-то заинтересует, дайте знать, я помогу.''')

[(u'_jobs', 0.31460292258060213),
 (u'career', 0.17843483191938694),
 (u'welcome', 0.10548385943118249),
 (u'_random_flood', 0.095441132554945046),
 (u'_general', 0.043160697662964993)]

In [278]:
predict_channel(u'а можно про оффхип по подробнее?')

[(u'theory_and_practice', 0.18742089350918822),
 (u'_random_flood', 0.17149058537958456),
 (u'deep_learning', 0.10809625622781312),
 (u'_meetings', 0.08422849061490377),
 (u'_jobs', 0.064569827995280271)]

In [279]:
predict_channel(u'вы hue не используете?')

[(u'big_data', 0.4067488429811868),
 (u'_random_flood', 0.13176042483251368),
 (u'_meetings', 0.069866712736964848),
 (u'theory_and_practice', 0.038138175807033971),
 (u'_general', 0.036084586040766996)]

In [280]:
predict_channel(u'А есть докер образы со всем что нужно на кэггле? ')

[(u'_random_flood', 0.17477553287773115),
 (u'theory_and_practice', 0.11370610353653217),
 (u'_general', 0.096752516918349366),
 (u'python', 0.075395133041503692),
 (u'deep_learning', 0.074907382919993187)]

In [281]:
predict_channel(u'а кто подскажет ответ на такой примитивный вопрос (смотрю в датасет bosch) - если я обнаруживаю две фичи с четкой линейной корреляцией - какая общая рекомендация что с ними делать? Выкинуть одну? Попытаться как-то извлечь информацию из второй? Что-то еще?')

[(u'theory_and_practice', 0.44678177339450326),
 (u'kaggle_crackers', 0.1769167991205596),
 (u'deep_learning', 0.049848548157015518),
 (u'_general', 0.033862639874370964),
 (u'python', 0.033203761366450653)]

In [282]:
predict_channel(u'ты его проходишь? ну и как последняя домашка? А по Visual Recognition чего-нибудь есть? Порекомендуйте')

[(u'edu_courses', 0.31219710104772791),
 (u'_random_flood', 0.071867109523670036),
 (u'python', 0.062321987276667608),
 (u'deep_learning', 0.058391603568720113),
 (u'visualization', 0.051522759150720111)]